# Solutions to the Assessment Problems for Computer Infrastructure Module

In [18]:
#import pandas
import pandas as pd
#import yfinance package
import yfinance as yf
#import datetime
import datetime as dt

## Problem 1: Data from yfinance

Create a function called `get_data` that takes the following parameters:
- `tickers`: A list of stock ticker symbols (e.g., ['AAPL', 'GOOG']).
- `start`: The start date for the data retrieval (default to 5 days ago from the current date).
- `end`: The end date for the data retrieval (default to the current date).
- `interval`: The data interval (default to '1h' for hourly data).

The function retrieves historical stock data for the specified tickers and date range using the yfinance package, and prints the first few rows of the retrieved data to the console.
The data is saved into a folder called *data* in the root of my repository to a CSV file using a filename with the format YYYYMMDD-HHmmss.csv.


For more  information on y-finance package, please visit the repository [link](https://github.com/ranaroussi/yfinance#history).
For more information on datetime package, please visit the documentation [link](https://docs.python.org/3/library/datetime.html).


In [19]:
# define function get_data()
def get_data(tickers, start = dt.datetime.now() - dt.timedelta(days=5), end = dt.datetime.now(), interval = "1h"):
    data = yf.download(tickers, start=start, end=end, interval=interval, auto_adjust=True)
    now= dt.datetime.now().strftime("%Y%m%d-%H%M%S")
    print(data.head())
    data.to_csv("data/" + now + ".csv", sep = ",")


Use the function to get data for the five FAANG stocks with the following tickers:
- META (Facebook)
- AAPL (Apple)
- AMZN (Amazon)
- NFLX (Netflix)
- GOOG (Google)

In [20]:
get_data(['META', 'AAPL', 'AMZN', 'NFLX' , 'GOOG'])

[*********************100%***********************]  5 of 5 completed

Price                           Close                                      \
Ticker                           AAPL        AMZN        GOOG        META   
Datetime                                                                    
2025-10-22 13:30:00+00:00  260.049896  217.449997  253.899994  732.340027   
2025-10-22 14:30:00+00:00  258.429901  218.739899  253.679993  729.630005   
2025-10-22 15:30:00+00:00  258.549988  218.970001  252.460007  730.020020   
2025-10-22 16:30:00+00:00  256.691406  217.800003  251.589996  727.169983   
2025-10-22 17:30:00+00:00  256.550995  217.304993  251.971100  728.927673   

Price                                         High                          \
Ticker                            NFLX        AAPL        AMZN        GOOG   
Datetime                                                                     
2025-10-22 13:30:00+00:00  1122.729980  262.850006  220.005005  257.079987   
2025-10-22 14:30:00+00:00  1117.420044  261.130005  218.960007  257.179

https://mimo.org/glossary/python/datetime

## Problem 2: Plotting Data

## Problem 3: Script

## Problem 4: Automation

## End